In [1]:
import arcpy
import pandas as pd
from arcgis.gis import GIS
import os

In [3]:
arcpy.env.workspace = "\\ParcelData_07_19_2022.gdb\\Parcels"
fc = arcpy.ListFeatureClasses()
# each county parcel is in WKID 3857

### Bring in prepared data

In [5]:
#start with a solar Resource Potential Basemap, then
# (1) Intersect the solar resource potential basemap with a feature class of California counties
# (2) Buffer this result by 1/2 mile
# (3) Because the original county and resource area maps are in WKID 3310, make a copy projected into WKID 3857 for intersection with full, original parcels data

solarRPB_3857 = "Parcelization_FullMethods.gdb\\TotalNewSolarRPB_IntCBuff_3857" #output coordinate system 3857
solarRPB_3310 = "Parcelization_FullMethods.gdb\\TotalNewSolarRPB_IntCBuff" #output coordinate system 3310
counties = "Parcelization_FullMethods.gdb\\Counties" #3310

# Parcels that overlap with more than 5 different APNs:
# overlaps = "Parcelization_FullMethods.gdb\MergeAllParcelizationD_v1_CountOverlapGT5"
# The dataset above was created by running the entire parcelization calculation (Part 1 and Part 2) once, and running the 'Count Overlapping Features' ArcGIS Pro tool on the final parcelization map result. 
# Any feature with 5 or more overlaps is identified in the dataset above ('overlaps'). These areas (parcels) are then erased during the main geoprocessing steps
# at the end of this script, where parcels needed to calculate parcelization within the resource potential basemap are grouped by county. 
# This step removes a very minor number of parcels, and results do not substatantially change by performing these steps.  


In [6]:
countySDF = pd.DataFrame.spatial.from_featureclass(counties)

### Create a dictionary of different naming conventions for identifying counties in datasets (ie 'ALPINE COUNTY' vs 'Alpine')

In [7]:
#first create a dictionary relating county names between datasets
countyList = []
countyNameOnly = {}
switchCtyNameOnly = {}
for i in range(countySDF.shape[0]):
    cntyName = countySDF.iloc[i].NAME
    countyList.append(cntyName)
    partsCN = cntyName.split(' ')[:-1]
    merge = ''
    
    for i in range(len(partsCN)):
        merge = merge+partsCN[i]
    merge = merge.upper()
    countyNameOnly[cntyName] = merge
    switchCtyNameOnly[merge] = cntyName
# write this dictionary that enables link between differenty ways of writing each county to a file to be read in part 2
pd.DataFrame.from_dict(switchCtyNameOnly,orient='index').to_excel('CountyList.xlsx') 

### Geoprocessing to create a dictionary relating every county to its neighboring county

In [9]:
# Select the county within the Solar Resource Potential Basemap Buffer/County Intersected dataset (SolarRPB_3310)
# Clip the original counties dataset by this buffered county (central county).  
# Get the names of the counties that were clipped by the buffered county (so it will be the main county, as well as any adjacent counties that are within 1/2 mile of a resource potential area)
# For each main/central county, record all the adjacent counties whose parcels will need to be included in the parcelization calculation for the main county
RPB_BufCty_Intersect = {}
for county in countyList:
    tmpOut = "FinalParcelizationMap.gdb\\tmpSel"
    whereClause = "NAME = "+"'"+county+ "'"
    arcpy.analysis.Select(solarRPB_3310, tmpOut, whereClause)
    
    tmpOut2 = "FinalParcelizationMap.gdb\\tmpClip"
    arcpy.analysis.Clip(counties,tmpOut,tmpOut2)
    
    allCounties = []
    for row in arcpy.da.SearchCursor(tmpOut2,['NAME']):
        allCounties.append(row[0])
    
    RPB_BufCty_Intersect[county] = allCounties
    
    
    
    
    

## Reduce parcels for each county to area neaded to perform parcelization calculation 
#### Includes areas within 1/2 mile buffer of neighboring counties 

In [12]:
baseParcels = "ParcelData_07_19_2022.gdb\\Parcels"
for county in countyList:
#     list of counties that border this county that has resource potential within 0.5 miles
#     These counties will be clipped and merged to create the parcelization footprint
    tmpOut = "FinalParcelizationMap.gdb\\tmpSel"
    whereClause = "NAME = "+"'"+county+ "'"
    arcpy.analysis.Select(solarRPB_3857, tmpOut, whereClause)  
    
    listMerge = RPB_BufCty_Intersect[county]
    
    storeTmpClips= []
    for i in range(len(listMerge)):
        cty = listMerge[i]
        #construct name of parcels feature class
        parcelsFC = baseParcels+"\\"+countyNameOnly[cty]+'_Parcels'
        
#         use buffered RPB from central county as clip on bordering counties
        tmpOutClip = "FinalParcelizationMap.gdb\\tmpClipP_"+ str(i)
        arcpy.analysis.Clip(parcelsFC,tmpOut,tmpOutClip)
        
        #Dissolve data by ParcelAPN
        tmpOutDiss = "FinalParcelizationMap.gdb\\tmpOutDiss_"+str(i)
        arcpy.management.Dissolve(tmpOutClip, tmpOutDiss,["PARCEL_APN"])

        #The next three lines are used in the second iteration of running the scripts when the overlapping parcels (greater than 5) will be removed
#         #erase data by count overlaps greater than 5
#         tmpOutErase = "FinalParcelizationMap.gdb\\tmpOutErase_"+str(i)
#         arcpy.analysis.Erase(tmpOutDiss, overlaps, tmpOutErase)
        
        storeTmpClips.append(tmpOutDiss) #change to storeTmpClips.append(tmpOutErase) when running a second time to remove the overlapping parcels
        
    #Merge all parcels together (all tmpOutClip) 
    outMerge = "Parcels_RPB.gdb\\"+countyNameOnly[county]+'_RPBParcels'
    addSourceInfo = "ADD_SOURCE_INFO"
    
    fieldmappings = arcpy.FieldMappings()
    
    for tmpFC in storeTmpClips:
        fieldmappings.addTable(tmpFC)
        
    fieldmap = arcpy.FieldMap()    
    for tmpFC in storeTmpClips:
        fieldmap.addInputField(tmpFC, "PARCEL_APN")
    
    arcpy.management.Merge(storeTmpClips, outMerge, fieldmappings, addSourceInfo)
    
    
        
    
    
        
    
    
    
    

Alameda County
Alpine County
Amador County
Butte County
Calaveras County
Colusa County
Contra Costa County
Del Norte County
El Dorado County
Fresno County
Glenn County
Humboldt County
Imperial County
Inyo County
Kern County
Kings County
Lake County
Lassen County
Los Angeles County
Madera County
Marin County
Mariposa County
Mendocino County
Merced County
Modoc County
Mono County
Monterey County
Napa County
Nevada County
Orange County
Placer County
Plumas County
Riverside County
Sacramento County
San Benito County
San Bernardino County
San Diego County
San Francisco County
San Joaquin County
San Luis Obispo County
San Mateo County
Santa Barbara County
Santa Clara County
Santa Cruz County
Shasta County
Sierra County
Siskiyou County
Solano County
Sonoma County
Stanislaus County
Sutter County
Tehama County
Trinity County
Tulare County
Tuolumne County
Ventura County
Yolo County
Yuba County
